In [19]:
import pandas as pd
import random
import json
from youtube_dl import YoutubeDL

download_dir = "outputs"

In [20]:
trailers = pd.read_csv("trailers.csv", index_col=0)
#trailers

In [21]:
trailer_yt_links = 'https://www.youtube.com/watch?v=' + trailers['youtube_key']
#trailer_yt_links

In [ ]:
ydl_opts = {
    'outtmpl': download_dir + '/%(title)s.%(ext)s',
    'format': 'bestvideo'
}
num_trailers_max = 100
with YoutubeDL(ydl_opts) as ydl:
    for t in trailer_yt_links[:num_trailers_max]:
        ydl.download([t])


[youtube] K_tLp7T6U1c: Downloading webpage
[youtube] K_tLp7T6U1c: Downloading embed webpage
[youtube] K_tLp7T6U1c: Refetching age-gated info webpage
[download] outputs/The Shawshank Redemption - Trailer.mp4 has already been downloaded
[download] 100% of 9.62MiB
[youtube] uMwECVb1q54: Downloading webpage
[youtube] uMwECVb1q54: Downloading embed webpage
[youtube] uMwECVb1q54: Refetching age-gated info webpage
[download] outputs/The Shawshank Redemption - Trailer.mp4 has already been downloaded
[download] 100% of 9.62MiB
[youtube] BXUEUwwgIyU: Downloading webpage
[youtube] BXUEUwwgIyU: Downloading video info webpage
[youtube] BXUEUwwgIyU: Downloading MPD manifest
[download] outputs/The Shawshank Redemption (1994) Re-Release Trailer.mp4 has already been downloaded
[download] 100% of 37.99MiB
[youtube] EsHlXpbMPOc: Downloading webpage
[youtube] EsHlXpbMPOc: Downloading video info webpage
[youtube] EsHlXpbMPOc: Downloading MPD manifest
[dashsegments] Total fragments: 25
[download] Destinatio